In [1]:
# required imports

import xlrd
import random
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt

sns.set_style('whitegrid')

In [ ]:
# Read the first sheet from the Excel File

file_name = '../DistrictWiseData.xlsm'
wb = xlrd.open_workbook(file_name)
sheetPM2p5 = wb.sheet_by_name('PM2.5')
sheetPM10 = wb.sheet_by_name('PM10')
sheetOzone = wb.sheet_by_name('Ozone')

In [ ]:
# Some pre-processing to prepare the DataFrame outof the excel file

print('Air Quality Reported During the Year 2019 From 231 Stations spread across 130 cities')
district_name = None
key = None
info_list = []
info = dict()

for i in range(sheet.nrows):
    numbers = list()
    for j in range(365):
        value = sheet.cell_value(i, j)
        if type(value) is str and not value.split():
            continue
        if value == 'SL':
            break
        if j == 0 and type(value) is str and 'Cases Reported during the Year' in value:
            state_name = value.replace(prefix, '').replace(suffix, '')
            info['state_name'] = state_name
            break
        if j == 1 and type(value) is str and value.split():
            key = value
            info[key] = None
        if type(value) is float:
            numbers.append(value)
    if numbers:
        info[key] = numbers[1:]
        n_keys = len(list(info.keys()))
        if n_keys == 16:
            info_list.append(info)
            info = dict()

keys = list(info_list[0].keys())
columns = keys.copy()
columns.insert(1,'Year')

final_list = list()

for il in info_list:
    year = 2001
    for i in range(15):
        temp_list = list()
        for k in keys[1:]:
            temp_list.append(il.get(k)[i])
        temp_list.insert(0, il.get('state_name').strip())
        temp_list.insert(1, year)
        year += 1
        final_list.append(temp_list)

In [ ]:
# preview the prepared DataFrame

df = pd.DataFrame(final_list, columns=columns)
df.head()

In [ ]:
# Keeping only the required columns

state_wise_total_crimes = df[['state_name', 'Total Crimes against Women', 'Year']]
data_for_map = state_wise_total_crimes[state_wise_total_crimes['Year']==2015]
data_for_map.head()

In [5]:
# reading the district wise shapefile of India in a GeoDataFrame and preview it

fp = "DistrictShapes.xlsx"
map_df = pd.read_excel(fp)
map_df.head()

,STATE,DISTRICT,MAPKEY,geometry
0,Andaman and Nicobar Islands,Nicobar,Andaman and Nicobar IslandsNicobar,"MULTIPOLYGON (((93.78772736000001 6.85264015, ..."
1,Andaman and Nicobar Islands,NorthandMiddleAndaman,Andaman and Nicobar IslandsNorthandMiddleAndaman,"MULTIPOLYGON (((92.93898009999999 12.22386074,..."
2,Andaman and Nicobar Islands,SouthAndaman,Andaman and Nicobar IslandsSouthAndaman,"MULTIPOLYGON (((92.47972107 10.52056026, 92.47..."
3,Andhra Pradesh,Anantapur,Andhra PradeshAnantapur,"POLYGON ((77.71420288 13.76079082, 77.7131424 ..."
4,Andhra Pradesh,Chittoor,Andhra PradeshChittoor,"POLYGON ((78.46292877 12.63537025, 78.4618988 ..."


In [6]:
# keep required columns and plot default map

map_df = map_df[['MAPKEY', 'geometry']]
map_df.plot()

TypeError: no numeric data to plot

In [ ]:
# read the district wise excel file and preview it

district_wise = pd.read_excel('district_wise.xls')
district_wise.head()

In [ ]:
# keeping only the required columns

district_wise = district_wise[['District/ Area', 'Total Crimes against Women']]

In [ ]:
# Join both the DataFrames by district names

merged = map_df.set_index('NAME_2').join(district_wise.set_index('District/ Area'))
merged.head()

In [ ]:
# NA Check

merged.isna().sum()

##### Seems like there are around 185 district names that are either mispelled or missing
##### I am not going to do correct these manually instead we will impute them with the mean (even though that will make our data incorrect but the propose of this notebook is to show you how to use GeoPandas to plot District wise stats on the Map of India)

In [ ]:
# Impute missing values by mean 

merged['Total Crimes against Women'].fillna(merged['Total Crimes against Women'].mean(), inplace=True)
merged.isna().sum()

In [ ]:
# summary

merged.describe()

In [ ]:
# create figure and axes for Matplotlib and set the title
fig, ax = plt.subplots(1, figsize=(10, 6))
ax.axis('off')
ax.set_title('District Wise Crime against women in India in 2015', fontdict={'fontsize': '25', 'fontweight' : '3'})

# plot the figure
merged.plot(column='Total Crimes against Women', cmap='YlOrRd', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

In [ ]:
# We save the output as a PNG image

fig.savefig("District_wise.png", dpi=100)

In [ ]:
# reading the district wise shapefile of India in a GeoDataFrame and preview it

fp = "gadm36_IND_shp/gadm36_IND_2.shp"
map_df = gpd.read_file(fp)
map_df.head()

In [ ]:
map_df = map_df[['NAME_1', 'NAME_2', 'geometry']]
map_df = map_df[map_df['NAME_1']=='Maharashtra']
map_df.plot()

In [ ]:
district_wise = pd.read_excel('district_wise.xls')
district_wise.head()

In [ ]:
# keeping only the required columns

district_wise = district_wise[['State/ UT', 'District/ Area', 'Total Crimes against Women']]
district_wise = district_wise[district_wise['State/ UT']=='Maharashtra']
district_wise.head()

In [ ]:
# Correct spellings of states from out dataframe to match those of GeoDataframe
# I corrected these names manually

district_wise['District/ Area'].iloc[0] = 'Ahmadnagar'


# 1449 is the sum of Amravati Commr and Amravati Rural from the district wise dataframe
district_wise = district_wise.append({'State/ UT': 'Maharashtra', 'District/ Area':'Amravati','Total Crimes against Women':1449.0}, ignore_index=True)

# 1449 is the sum of Aurangabad Commr and Aurangabad Rural from the district wise Dataframe
# same is done for rest of the cities
district_wise = district_wise.append({'State/ UT': 'Maharashtra', 'District/ Area':'Aurangabad','Total Crimes against Women':1421.0}, ignore_index=True)

district_wise['District/ Area'].iloc[6] = 'Bid'

district_wise['District/ Area'].iloc[8] = 'Buldana'

district_wise['District/ Area'].iloc[11]  = 'Garhchiroli'

district_wise['District/ Area'].iloc[12]  = 'Gondiya'

district_wise['District/ Area'].iloc[18] = 'Mumbai City'

district_wise['District/ Area'].iloc[19] = 'Mumbai Suburban'

district_wise = district_wise.append({'State/ UT': 'Maharashtra', 'District/ Area':'Nagpur','Total Crimes against Women':1754.0}, ignore_index=True)

district_wise = district_wise.append({'State/ UT': 'Maharashtra', 'District/ Area':'Nashik','Total Crimes against Women':1416.0}, ignore_index=True)

district_wise = district_wise.append({'State/ UT': 'Maharashtra', 'District/ Area':'Pune','Total Crimes against Women':1993.0}, ignore_index=True)

district_wise['District/ Area'].iloc[34] = 'Raigarh'

district_wise = district_wise.append({'State/ UT': 'Maharashtra', 'District/ Area':'Thane','Total Crimes against Women':2289.0}, ignore_index=True)

district_wise = district_wise.append({'State/ UT': 'Maharashtra', 'District/ Area':'Solapur','Total Crimes against Women':874.0}, ignore_index=True)

In [ ]:
# Join both the DataFrames by district names

merged = map_df.set_index('NAME_2').join(district_wise.set_index('District/ Area'))
merged.head()

In [ ]:
# NA Check

merged.isna().sum()

In [ ]:
# summary

merged.describe()

In [ ]:
# create figure and axes for Matplotlib and set the title
fig, ax = plt.subplots(1, figsize=(10, 6))
ax.axis('off')
ax.set_title('District Wise Crime against women in Maharastra in 2015', fontdict={'fontsize': '25', 'fontweight' : '3'})

# plot the figure
merged.plot(column='Total Crimes against Women', cmap='YlOrRd', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

In [ ]:
# Saving the output as a PNG image

fig.savefig("Maharashtra.png", dpi=600)